In [1]:
import os
import pandas as pd
import numpy as np
from sklearn.linear_model import LinearRegression
from sklearn.metrics import root_mean_squared_error, r2_score
from ticker_download_manager import TickerDownloadManager

## Assemble dataframes of predictors and targets

In [2]:
n_years = 2
trading_days_per_year = 252
predictor_tickers = ["I:SPX", "I:DJI", "I:COMP", "I:NDX"]
tdm_predictors_path = os.path.join("input", "annual_predictors")
tdm_predictors = TickerDownloadManager(
    download_folder_name=tdm_predictors_path, tickers=predictor_tickers
)
long_predictors_df, _, _ = tdm_predictors.get_latest_tickers(
    days_in_past=(trading_days_per_year * n_years), use_cache=True
)
target_tickers = ["GILD", "SBUX", "NVDA"]
tdm_targets_path = os.path.join("input", "annual_targets")
tdm_responses = TickerDownloadManager(
    download_folder_name=tdm_targets_path, tickers=target_tickers
)
long_targets_df, _, _ = tdm_responses.get_latest_tickers(
    days_in_past=(trading_days_per_year * n_years), use_cache=True
)

In [3]:
long_predictors_df

,datetime,ticker,open,high,low,close,volume,vwap,transactions
0,2023-05-24 23:59:59,I:SPX,4132.960000,4132.960000,4103.980000,4115.240000,NaN,NaN,NaN
1,2023-05-24 23:59:59,I:NDX,13589.018099,13655.733726,13520.920309,13604.482463,NaN,NaN,NaN
2,2023-05-24 23:59:59,I:COMP,12481.894539,12529.632773,12415.851994,12484.162036,NaN,NaN,NaN
3,2023-05-24 23:59:59,I:DJI,33021.760000,33031.750000,32752.440000,32799.920000,NaN,NaN,NaN
4,2023-05-25 23:59:59,I:SPX,4155.710000,4165.740000,4129.730000,4151.280000,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...
2024,2025-05-29 23:59:59,I:DJI,42190.020000,42266.000000,41828.350000,42215.730000,NaN,NaN,NaN
2025,2025-05-30 23:59:59,I:DJI,42192.350000,42376.080000,41906.160000,42270.070000,NaN,NaN,NaN
2026,2025-05-30 23:59:59,I:COMP,19131.218805,19157.782160,18847.744022,19113.766523,NaN,NaN,NaN
2027,2025-05-30 23:59:59,I:SPX,5903.670000,5922.140000,5843.660000,5911.690000,NaN,NaN,NaN


In [4]:
long_targets_df

,datetime,ticker,open,high,low,close,volume,vwap,transactions
0,2023-05-24 23:59:59,GILD,79.120,79.2600,78.1400,79.050,5.863905e+06,78.8945,60021
1,2023-05-24 23:59:59,NVDA,30.210,30.6070,29.8060,30.538,7.214185e+08,32.1595,1002242
2,2023-05-24 23:59:59,SBUX,100.800,100.8100,99.1100,99.610,6.134820e+06,99.6382,106380
3,2023-05-25 23:59:59,GILD,77.470,78.1800,76.2600,77.650,8.118148e+06,77.3649,80739
4,2023-05-25 23:59:59,NVDA,38.523,39.4800,36.6350,37.980,1.543834e+09,38.3018,2225510
...,...,...,...,...,...,...,...,...,...
1513,2025-05-29 23:59:59,GILD,108.590,111.2400,108.1200,111.110,5.597110e+06,110.6354,82440
1514,2025-05-29 23:59:59,SBUX,84.900,85.7007,83.6990,84.050,1.031255e+07,84.4479,128709
1515,2025-05-30 23:59:59,SBUX,83.940,84.2700,83.1933,83.950,7.402691e+06,83.8235,93699
1516,2025-05-30 23:59:59,GILD,111.620,111.6200,109.5795,110.080,1.197651e+07,110.1225,84784


In [5]:
long_predictors_df["datetime"] = pd.to_datetime(long_predictors_df["datetime"])
long_predictors_df.sort_values(by=["datetime", "ticker"], inplace=True)
long_predictors_df.set_index("datetime", inplace=True)
wide_predictors_df = long_predictors_df[["ticker", "close"]].pivot(
    columns="ticker", values="close"
)
wide_predictors_df

ticker,I:COMP,I:DJI,I:NDX,I:SPX
datetime,,,,
2023-05-24 23:59:59,12484.162036,32799.92,13604.482463,4115.24
2023-05-25 23:59:59,12698.093678,32764.65,13938.530838,4151.28
2023-05-26 23:59:59,12975.687543,33093.34,14298.408714,4205.45
2023-05-30 23:59:59,13017.425569,33042.78,14354.986054,4205.52
2023-05-31 23:59:59,12935.285372,32908.27,14254.090248,4179.83
...,...,...,...,...
2025-05-23 23:59:59,18737.207177,41603.07,20915.654860,5802.82
2025-05-27 23:59:59,19199.163247,42343.65,21414.990676,5921.54
2025-05-28 23:59:59,19100.937701,42098.70,21318.168484,5888.55


In [6]:
long_targets_df["datetime"] = pd.to_datetime(long_targets_df["datetime"])
long_targets_df.sort_values(by=["datetime", "ticker"], inplace=True)
long_targets_df.set_index("datetime", inplace=True)
wide_targets_df = long_targets_df[["ticker", "close"]].pivot(
    columns="ticker", values="close"
)
wide_targets_df

ticker,GILD,NVDA,SBUX
datetime,,,
2023-05-24 23:59:59,79.05,30.538,99.61
2023-05-25 23:59:59,77.65,37.980,98.44
2023-05-26 23:59:59,77.09,38.946,98.53
2023-05-30 23:59:59,76.33,40.111,97.75
2023-05-31 23:59:59,76.94,37.834,97.64
...,...,...,...
2025-05-23 23:59:59,107.37,131.290,84.40
2025-05-27 23:59:59,109.09,135.500,87.01
2025-05-28 23:59:59,108.44,134.810,86.00


## Convert everything to log returns

In [7]:
predictors_log_returns_df = np.log(wide_predictors_df / wide_predictors_df.shift(1))[1:]
predictors_log_returns_df

ticker,I:COMP,I:DJI,I:NDX,I:SPX
datetime,,,,
2023-05-25 23:59:59,0.016991,-0.001076,0.024258,0.008720
2023-05-26 23:59:59,0.021626,0.009982,0.025491,0.012965
2023-05-30 23:59:59,0.003211,-0.001529,0.003949,0.000017
2023-05-31 23:59:59,-0.006330,-0.004079,-0.007053,-0.006127
2023-06-01 23:59:59,0.012728,0.004648,0.013063,0.009806
...,...,...,...,...
2025-05-23 23:59:59,-0.010011,-0.006135,-0.009366,-0.006731
2025-05-27 23:59:59,0.024355,0.017645,0.023593,0.020253
2025-05-28 23:59:59,-0.005129,-0.005802,-0.004531,-0.005587


In [8]:
target_log_returns_df = np.log(wide_targets_df / wide_targets_df.shift(1))[1:]
target_log_returns_df

ticker,GILD,NVDA,SBUX
datetime,,,
2023-05-25 23:59:59,-0.017869,0.218088,-0.011815
2023-05-26 23:59:59,-0.007238,0.025116,0.000914
2023-05-30 23:59:59,-0.009908,0.029475,-0.007948
2023-05-31 23:59:59,0.007960,-0.058442,-0.001126
2023-06-01 23:59:59,-0.009533,0.049905,-0.001230
...,...,...,...
2025-05-23 23:59:59,0.005885,-0.011661,0.005108
2025-05-27 23:59:59,0.015892,0.031563,0.030456
2025-05-28 23:59:59,-0.005976,-0.005105,-0.011676


## Train regressions on every target using all predictors

### Explanatory predicting same-day close

In [9]:
n_train = int(len(predictors_log_returns_df) * 0.75)
explanatory_dfs = {}
for target_ticker in target_log_returns_df.columns:
    print(f"********** Modeling {target_ticker} *********")
    np.random.seed(123)
    Xy = predictors_log_returns_df.merge(
        target_log_returns_df[target_ticker], left_index=True, right_index=True
    ).dropna()
    X = Xy[predictor_tickers]
    y = Xy[target_ticker]
    X_train = X.iloc[:n_train, :]
    y_train = y[:n_train]
    X_test = X.iloc[n_train:, :]
    y_test = y[n_train:]
    model = LinearRegression()
    model.fit(X_train, y_train)
    y_pred = model.predict(X_test)
    explanatory_df = X_test.copy()
    explanatory_df[target_ticker] = y_test
    explanatory_df[f"{target_ticker} % Return"] = (np.exp(y_test) - 1) * 100
    explanatory_df[f"{target_ticker}_pred"] = y_pred
    explanatory_df[f"{target_ticker}_pred % Return"] = (np.exp(y_pred) - 1) * 100
    explanatory_dfs[target_ticker] = explanatory_df
    rmse = root_mean_squared_error(y_test, y_pred)
    rmse_pct = (np.exp(rmse) - 1) * 100
    r2 = r2_score(y_test, y_pred)
    print(f"RMSE: {rmse:.4f} ({rmse_pct:.4f}%)")
    print(f"R2: {r2:.4f}")
    print(f"Coeffcients: {list(zip(wide_predictors_df.columns, model.coef_))}")
    print(f"Intercept: {model.intercept_:.4f}")
explanatory_models_filename = os.path.join(
    "output", "returns_regression", "explanatory_models.xlsx"
)
with pd.ExcelWriter(explanatory_models_filename, engine="openpyxl") as writer:
    for sheet_name, df in explanatory_dfs.items():
        df.to_excel(writer, sheet_name=sheet_name, index=False)

********** Modeling GILD *********
RMSE: 0.0165 (1.6601%)
R2: 0.2022
Coeffcients: [('I:COMP', np.float64(-0.2341598155472541)), ('I:DJI', np.float64(1.0634523145205708)), ('I:NDX', np.float64(-0.5247261487416185)), ('I:SPX', np.float64(0.34096054454300434))]
Intercept: -0.0000
********** Modeling NVDA *********
RMSE: 0.0215 (2.1731%)
R2: 0.7162
Coeffcients: [('I:COMP', np.float64(1.514223684321716)), ('I:DJI', np.float64(-2.4177387841554143)), ('I:NDX', np.float64(0.1613772026719718)), ('I:SPX', np.float64(1.9377899327296966))]
Intercept: 0.0019
********** Modeling SBUX *********
RMSE: 0.0187 (1.8834%)
R2: 0.4593
Coeffcients: [('I:COMP', np.float64(2.093569801686531)), ('I:DJI', np.float64(-0.13217154415547516)), ('I:NDX', np.float64(-1.9462164143076537)), ('I:SPX', np.float64(1.0562900932127868))]
Intercept: -0.0010


### Predicting *next* day's close

First explore the necessary DataFrame shift.

In [10]:
shift_me = pd.DataFrame({"col0": [1, 2, 3, 4], "col1": [1, 2, 3, 4]})
shift_me["col1"] = shift_me["col1"].shift(-1)
shift_me

,col0,col1
0,1,2.0
1,2,3.0
2,3,4.0
3,4,NaN


Then perform the modeling. It isn't a huge code change

In [11]:
n_train = int(len(predictors_log_returns_df) * 0.75)
predictive_dfs = {}
for target_ticker in target_log_returns_df.columns:
    print(f"********** Modeling {target_ticker} *********")
    np.random.seed(123)
    Xy = predictors_log_returns_df.merge(
        target_log_returns_df[target_ticker], left_index=True, right_index=True
    ).dropna()
    Xy[target_ticker] = Xy[target_ticker].shift(-1)
    Xy.dropna(inplace=True)
    X = Xy[predictor_tickers]
    y = Xy[target_ticker]
    X_train = X.iloc[:n_train, :]
    y_train = y[:n_train]
    X_test = X.iloc[n_train:, :]
    y_test = y[n_train:]
    model = LinearRegression()
    model.fit(X_train, y_train)
    y_pred = model.predict(X_test)
    predictive_df = X_test.copy()
    predictive_df[target_ticker] = y_test
    predictive_df[f"{target_ticker} % Return"] = (np.exp(y_test) - 1) * 100
    predictive_df[f"{target_ticker}_pred"] = y_pred
    predictive_df[f"{target_ticker}_pred % Return"] = (np.exp(y_pred) - 1) * 100
    predictive_dfs[target_ticker] = predictive_df
    rmse = root_mean_squared_error(y_test, y_pred)
    rmse_pct = (np.exp(rmse) - 1) * 100
    r2 = r2_score(y_test, y_pred)
    print(f"RMSE: {rmse:.4f} ({rmse_pct:.4f}%)")
    print(f"R2: {r2:.4f}")
    print(f"Coeffcients: {list(zip(wide_predictors_df.columns, model.coef_))}")
    print(f"Intercept: {model.intercept_:.4f}")
predcitive_dfs_filename = os.path.join(
    "output", "returns_regression", "predictive_dfs.xlsx"
)
with pd.ExcelWriter(predcitive_dfs_filename, engine="openpyxl") as writer:
    for sheet_name, df in predictive_dfs.items():
        df.to_excel(writer, sheet_name=sheet_name, index=False)

********** Modeling GILD *********
RMSE: 0.0185 (1.8665%)
R2: -0.0036
Coeffcients: [('I:COMP', np.float64(0.013739771140750982)), ('I:DJI', np.float64(0.26189720870902644)), ('I:NDX', np.float64(0.5270873753415982)), ('I:SPX', np.float64(-0.6335147347242686))]
Intercept: 0.0002
********** Modeling NVDA *********
RMSE: 0.0408 (4.1635%)
R2: -0.0148
Coeffcients: [('I:COMP', np.float64(0.5641495174167093)), ('I:DJI', np.float64(-0.029275739204457873)), ('I:NDX', np.float64(-1.0807510004262348)), ('I:SPX', np.float64(0.6896834708340537))]
Intercept: 0.0033
********** Modeling SBUX *********
RMSE: 0.0254 (2.5678%)
R2: 0.0088
Coeffcients: [('I:COMP', np.float64(0.7417534014653773)), ('I:DJI', np.float64(-0.35012491598291307)), ('I:NDX', np.float64(0.504747148930115)), ('I:SPX', np.float64(-0.8710257013095362))]
Intercept: -0.0001
